In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive')
!pwd

/content/drive/My Drive


In [3]:
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [4]:
import pandas as pd
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
# %matplotlib inline

import keras
from keras import optimizers
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
from keras.layers.advanced_activations import ReLU
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
import random
import math
import matplotlib.image as mpimg
import cv2
import os

In [5]:
# path = 'train_dataset'

In [6]:
# for img in os.listdir(path):  # iterate over each image per dogs and cats
#     img_array = cv2.imread(os.path.join(path,img))
#     img_array = cv2.resize(img_array, (216, 216),interpolation = cv2.INTER_AREA)


In [7]:
# img_array.shape

In [8]:
# target_path = 'target_image'

In [9]:
# for img in os.listdir(target_path):  # iterate over each image per dogs and cats
#     target_arr = cv2.imread(os.path.join(target_path,img))
#     target_arr=cv2.resize(target_arr, (216, 216),interpolation = cv2.INTER_AREA)

In [10]:
# target_arr

In [11]:
def load_dataset():
  train_data = pd.read_csv('train_aug_dataset.csv')
  y_train = pd.read_csv('augmented_labels_delta.csv')
  target_data = pd.read_csv('target_data.csv')
  # target_label = pd.read_csv('target_label.csv')
  return train_data, y_train, target_data

In [12]:
train_data, y_train, target_data = load_dataset()

In [126]:
# train_data = img_array
# target_data = target_arr

In [13]:
num_classes = 4
train_data = np.array(train_data.iloc[:,:])
train_data = np.array([np.reshape(i,(128,128)) for i in train_data])
target_data = np.array(target_data.iloc[:,:])
target_data = np.array([np.reshape(i,(128,128)) for i in target_data])

In [15]:
train_data.shape
target_data.shape

(928, 128, 128)

In [16]:
train_data = train_data.astype('float32')
train_data = train_data/255

rgb_batch = np.repeat(train_data[..., np.newaxis], 3, -1)
rgb_batch.shape

target_data = target_data.astype('float32')
target_data = target_data/255

rgb_target = np.repeat(target_data[..., np.newaxis], 3, -1)
rgb_target.shape

(928, 128, 128, 3)

In [17]:
target_data = rgb_target

In [18]:
target_data.shape

(928, 128, 128, 3)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(rgb_batch, y_train, test_size = 0.3, random_state=666)
target_xtrain, target_xtest = train_test_split(target_data, test_size = 0.3, random_state=666)

In [20]:
input_shape = (128, 128, 3)

In [ ]:
# print(y_test)

In [50]:
def VGG_16(input_image):
    model = Conv2D(64, (3,3), padding="same", input_shape = input_shape)(input_image)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(64, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    # model = Dropout(0.25)(model)

    model = Conv2D(128, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(128, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    # model = Dropout(0.25)(model)

    # model = Conv2D(256, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = Conv2D(256, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)


    return model

In [51]:
current_input = Input(shape=input_shape)
current_model = VGG_16(current_input)

target_input = Input(shape=input_shape)
target_model = VGG_16(target_input)

In [52]:
conv = concatenate([current_model, target_model])

In [53]:
conv = Flatten()(conv)
dense = Dense(512)(conv)
dense = ReLU(max_value=None)(dense)
dense = Dropout(0.5)(dense)
# dense = Dense(512)(dense)
# dense = ReLU(max_value=1.0)(dense)
# dense = Dropout(0.5)(dense)

output = Dense(num_classes, activation='linear')(dense)
# output = math.floor(output)
model = Model(inputs=[current_input, target_input], outputs=[output])

opt = Adam(learning_rate=0.0001)

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])


In [54]:
best_weights_file="weights.aug.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_mse', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

In [55]:
model.fit([x_train, target_xtrain], y_train,
          epochs=100,
          validation_split=0.3,
          callbacks = callbacks,
          verbose=1)

Epoch 1/100
15/15 [==============================] - 3s 133ms/step - loss: 187.2904 - mse: 187.2904 - val_loss: 183.7189 - val_mse: 183.7189

Epoch 00001: val_mse improved from -inf to 183.71893, saving model to weights.aug.best.hdf5
Epoch 2/100
15/15 [==============================] - 2s 121ms/step - loss: 183.5077 - mse: 183.5077 - val_loss: 178.9776 - val_mse: 178.9776

Epoch 00002: val_mse did not improve from 183.71893
Epoch 3/100
15/15 [==============================] - 2s 117ms/step - loss: 186.8622 - mse: 186.8622 - val_loss: 169.6604 - val_mse: 169.6604

Epoch 00003: val_mse did not improve from 183.71893
Epoch 4/100
15/15 [==============================] - 2s 118ms/step - loss: 169.8379 - mse: 169.8379 - val_loss: 162.5482 - val_mse: 162.5482

Epoch 00004: val_mse did not improve from 183.71893
Epoch 5/100
15/15 [==============================] - 2s 118ms/step - loss: 171.6377 - mse: 171.6377 - val_loss: 162.2836 - val_mse: 162.2836

Epoch 00005: val_mse did not improve from 

In [56]:
model.evaluate([x_test, target_xtest], y_test)

9/9 [==============================] - 0s 36ms/step - loss: 120.9053 - mse: 120.9053


[120.90534210205078, 120.90534210205078]

In [57]:
prediction = model.predict([x_test, target_xtest])
prediction = np.rint(prediction)

In [58]:
print(prediction)

[[ 33. -10.   0.   6.]
 [ -0.  -0.  -0.   0.]
 [ -0.  -0.  -0.   0.]
 ...
 [-10.  -3.  -6.   9.]
 [ -0.  -0.  -0.   0.]
 [  3.   1.  -4.   2.]]


In [59]:
np.savetxt('predictions.csv', prediction, delimiter=",")

In [61]:
y_test.to_numpy()

array([[ 35, -13,   0,   7],
       [  0,   0,   0,   0],
       [  0,   0,   0,   0],
       ...,
       [-25, -18,   0,  -3],
       [  0,   0,   0,   0],
       [ 25, -18,   0,   2]])

In [62]:
np.savetxt('ground_truth.csv', y_test, delimiter=',')

In [ ]:
# import cv2
# from google.colab.patches import cv2_imshow
# img1 = cv2.imread('data65_27.png')
# img2 = cv2.imread('target_image_1.png')
#mg1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# img1 = cv2.resize(img1, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img1)
# img1.shape

# img1 = np.expand_dims(img1, axis = 0)
# img1 = np.expand_dims(img1, axis = 3)
# img1 = img1.astype('float32')
# img1 = img1/255
# plt.imshow(img1.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
# from PIL import Image
# # Open the image form working directory
# img1 = Image.open('/content/data65_27.png')
# img2 = Image.open('/content/target_image_1.png')

In [ ]:
# from matplotlib import image
# from matplotlib import pyplot
# # load image as pixel array
# img1 = image.imread('/content/data65_27.png')
# img2 = image.imread('/content/target_image_1.png')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('/content/data65_27.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img= cv2.resize(img, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img.shape

img = np.reshape(img, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img = img/255
img = np.repeat(img[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img.shape

In [ ]:
img2 = cv2.imread('/content/target_image_1.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img2= cv2.resize(img2, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img2.shape

img2 = np.reshape(img2, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img2 = img2/255
img2 = np.repeat(img2[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img2.shape

In [ ]:
predictionimg = model.predict([img,img2])

In [ ]:
predictionimg